![ups logo](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Logo_Universidad_Polit%C3%A9cnica_Salesiana_del_Ecuador.png/640px-Logo_Universidad_Polit%C3%A9cnica_Salesiana_del_Ecuador.png)

# Inteligencia Artificial

# Practica 03

# Transformaciones, KNN vs Redes Neuronales

### Autor: Diego Tapia


## Descripción del dataset

Nombre del dataset: Statlog (German Credit Data). Enlace: http://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data) [1]

Descripción general: Este conjunto de datos clasifica a las personas descritas por un conjunto de atributos como riesgos crediticios buenos o malos.

Número de Variables (o atributos): 21

Número de instancias (clientes en el banco): 1000

**Salida**:

TIPOCLIENTE (BUENO:1, MALO:2)

### Importacion de librerias

In [1]:
# Importacion de librerias

%pip install scikeras

import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt 

from time import time
from datetime import datetime, timedelta

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, MinMaxScaler
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import Pipeline
import pickle
import seaborn as sns
import copy

from sklearn.model_selection import train_test_split
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from keras.layers import Dropout

import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.models import model_from_json
from keras.models import load_model

from sklearn import metrics 
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn import set_config   

print("Módulos y clases importados")

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


Módulos y clases importados


### Carga del dataset



In [3]:
nombresVariables=['ESTADOCUENTACORRIENTE','PLAZOMESESCREDITO','HISTORIALCREDITO','PROPOSITOCREDITO','MONTOCREDITO',
                  'SALDOCUENTAAHORROS','TIEMPOACTUALEMPLEO','TASAPAGO','ESTADOCIVILYSEXO','GARANTE','TIEMPORESIDENCIAACTUAL',
                  'ACTIVOS','EDAD','OTROSPLANESPAGO','VIVIENDA','CANTIDADCREDITOSEXISTENTES','EMPLEO',
                  'CANTIDADPERSONASAMANTENER','TELEFONO','TRABAJADOREXTRANJERO','TIPOCLIENTE']
#Para cargar un dataframe desde una URL
dfOriginal = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data', sep = ' ', 
                         names=nombresVariables)

dataframe=copy.deepcopy(dfOriginal)
print('cantidad de observaciones (clientes): ',dataframe.shape[0])
print('cantidad de variables: ',dataframe.shape[1])
print(dataframe.shape)
dataframe.head()

cantidad de observaciones (clientes):  1000
cantidad de variables:  21
(1000, 21)


,ESTADOCUENTACORRIENTE,PLAZOMESESCREDITO,HISTORIALCREDITO,PROPOSITOCREDITO,MONTOCREDITO,SALDOCUENTAAHORROS,TIEMPOACTUALEMPLEO,TASAPAGO,ESTADOCIVILYSEXO,GARANTE,...,ACTIVOS,EDAD,OTROSPLANESPAGO,VIVIENDA,CANTIDADCREDITOSEXISTENTES,EMPLEO,CANTIDADPERSONASAMANTENER,TELEFONO,TRABAJADOREXTRANJERO,TIPOCLIENTE
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2


## 1 Descripción de variables del dataset

|Id	|Variable	|Descripción breve	|Tipo	|Técnica de transformación a aplicar|
----|------------|-------------------|------------|------------------------------|
|1	|ESTADOCUENTACORRIENTE	|Indica si existe o no una cuenta corriente|Categórica ordinal|Codificación binaria|
|2	|PLAZOMESESCREDITO |Duracion del prestamo en meses|	Numérica discreta	|Estandarización|
|3	|HISTORIALCREDITO |Historial de credito; sin credito, retraso en pagos, creditos pagados|Categórica ordinal (en el contexto del banco)|	Codificación binaria|
|4  |PROPOSITOCREDITO |Proposito del credito: auto nuevo, educacion, reparaciones, negocios, etc.|Categórica nominal| Codificación binaria|
|5  |MONTOCREDITO |Cantidad del credito|Numérica continua| Estandarización|
|6  |SALDOCUENTAAHORROS |Ahorros en la cuenta o límites; <$100, $100<X<$500, $1000>,desconocido |Categórica ordinal| Codificación binaria|
|7  |TIEMPOACTUALEMPLEO |Indica el tiempo  que trabaja el cliente o el estado; desempleado, < 1 año, 1año<X<4años, > 7años |Categórica ordinal| Codificación binaria|
|8  |TASAPAGO |Tasa de pago a plazos en porcentaje del ingreso disponible |Numérica continua| Estandarización|
|9  |ESTADOCIVILYSEXO |Indica el estado civil y sexo; hombre:divorciado, mujer:soltera: hombre:casado, ... |Categórica ordinal (en el contexto del banco)| Codificación binaria|
|10 |GARANTE |Indica si tiene codeudores o garantes; ninguno, codeudor, garante |Categórica nominal|Codificación binaria|
|11 |TIEMPORESIDENCIAACTUAL |Tiempo de residencia en años |Numérica discreta| Estandarización|
|12 |ACTIVOS |activos o bienes del cliente; desconocidos, autos ,sin propiedad,bienes raices, etc. |Categórica ordinal (en el contexto del banco)| Codificación binaria|
|13 |EDAD |Edad del cliente en años |Numérica discreta| Estandarización|
|14 |OTROSPLANESPAGO |Otros planes de pago del cliente; banco, tiendas, ninguno |Categórica nominal| Se elimina la variable, no se considera relevante para este análisis|
|15 |VIVIENDA |Tipo de vivienda; rentada, propia, cedida |Categórica ordinal (en el contexto del banco)| Codificación binaria|
|16 |CANTIDADCREDITOSEXISTENTES |Número de préstamos exisentes en el banco |Numérica discreta| Estandarización|
|17 |EMPLEO  |Indica el estado de empleo del cliente; desempleado, trabajador no calificado,trabajador calificado, administrativo, autonomo, etc.  |Categórica ordinal (en el contexto del banco)| Codificación binaria|
|18 |CANTIDADPERSONASAMANTENER |Número de personas de la que es responsable el cliente.  |Numérica discreta| Estandarización|
|19 |TELEFONO |Indica si se tiene o no registrado el numero del cliente; SI, NO  |Categórica nominal| Se elimina la variable, no se considera relevante para este análisis|
|20 |TRABAJADOREXTRANJERO |Indica si es trabajador extranjero o no; SI, NO  |Categórica nominal| Codificación binaria|
|21 |**TIPOCLIENTE** |**Variable objetivo.** Indica si es un cliente Bueno o Malo para el credito; 1: Bueno, 0: Malo  |Numérica discreta| Codificación binaria|

